# Un scénario complet

Quelques informations et paramètres pour commencer 

In [1]:
from Entete import *

ImportError: No module named 'Entete'

Tous les includes

In [2]:
#Param_SBMLfile = "/Users/jeremiebourdon/Downloads/Hep-G2.xml"

In [3]:
## DEBUG ONLY
begintotal = timer()
eprint("Testing",Param_SBMLfile,"for",Param_Drug)

## END DEBUG ONLY

NameError: name 'timer' is not defined

Chargement des listes de correspondances

In [4]:
#from AllTables import *
from AllTables import *

ImportError: No module named 'AllTables'

In [5]:
Param_Pourcentage = 0.95

Construction de la liste des cibles

In [6]:
# étape 1 : construction du fichier d'association Reactions <-> EC numbers

# obsolete : La liste RXN -> liste gènes est désormais récupérée directement depuis HMRDatabase

# dictionnaire HMR_to_Ensembl



eprint("Entering : Association RxN <-> Enzymes list construction")

SBMLfile = Param_SBMLfile

#SBMLfile = raw_input("Enter the SBML filename ? ")

def ConstructAssociationList(SBMLfile, RXN_EC_file="/tmp/RXN_EC.txt"):

    import os
    os.system("../scripts/Construct_List_RXN_Enzymes.sh "+SBMLfile+" "+RXN_EC_file)
    fd = open(RXN_EC_file,"r")
    lines = fd.readlines()

    genes = dict()
    for l in lines:
        t=l.split(" ")
        cum=''
        for i in range(1,len(t)-1):
            cum=cum+t[i]+" "
        genes[t[0]]=cum
    return genes

genes=ConstructAssociationList(SBMLfile,RXN_EC_file)


NameError: name 'eprint' is not defined

In [7]:
# Etape 2 : Single Reaction Analysis

eprint("Entering : Single Reaction Analysis")

## DEBUG ONLY
begin = timer()
## END DEBUG ONLY


# perfoming a single_reaction_deletion analysis of the network

model = cobra.io.read_sbml_model(Param_SBMLfile)

# the objective coefficient must be properly set
model.reactions.CancerBiomass_OF.objective_coefficient=1

# reference biomass growth
fba=model.optimize().f

# single reaction deletion analysis


if (Param_DoReconstruction and Param_Pourcentage<1):
    rates, status = cobra.flux_analysis.single_reaction_deletion(model,solver=whichsolver)
else:
    rates=dict()
    for r in model.reactions:
        rates[r.id]=0
        
## DEBUG ONLY
end = timer()
eprint("Execution time (Single Reaction Analysis) = ",end-begin,"seconds")
## END DEBUG ONLY

Entering : Single Reaction Analysis
Execution time (Single Reaction Analysis) =  40.433124470999246 seconds


In [8]:
# Etape 3 : Extraction des enzymes régulateurs (nécessite étape 1 et 2)

def getRegulators(rates, percentage=1.1):
    regulators = dict()

    cpt=0
    for r in rates:
        if (rates.get(r)<percentage*fba):
            if (HMR_to_Ensembl.get(r)!=''):
                # print(r,':',rates.get(r)," ",genes.get(r))
                cpt=cpt+1
                l=HMR_to_Ensembl.get(r)
                if (l):
                    for i in l:
                        if (i != ''):
                            regulators[i]=1
    return regulators


def getRegulators_obsolete(rates, percentage=1.1):
    regulators = dict()

    cpt=0
    for r in rates:
        if (rates.get(r)<percentage*fba):
            if (genes.get(r)!=''):
                # print(r,':',rates.get(r)," ",genes.get(r))
                cpt=cpt+1
                l=genes.get(r).split(' ')
                for i in l:
                    if (i != ''):
                        regulators[i]=1
    return regulators

if (Param_DoReconstruction):
    regulators = getRegulators(rates, Param_Pourcentage)

In [9]:
# Etape 4 : récupération de la liste de gènes régulateurs
def GeneListFromEC(ecnumber):
    if (ecnumber.find('-')>0):
        cum=list()
        for i in range(1,500):
            gl=ECTCDBToEnsembl.get(ecnumber.replace('-',str(i)))
            if (gl != None and len(gl)>0):
                cum+=gl
        return cum
    else:
        gl=ECTCDBToEnsembl.get(ecnumber)
        if (gl == None):
            gl=list()
        return gl

if (Param_DoReconstruction):
    TargetGenesEnsembl=list()
    for ec in regulators:
#        TargetGenesEnsembl+=GeneListFromEC(ec.lower())
        TargetGenesEnsembl+=[ec.replace("'","%27")]
    
    TargetGenes = [EnsemblToGeneNames.get(x) for x in TargetGenesEnsembl]
    TargetGenes=list()
    for x in TargetGenesEnsembl:
        if (genelist.get(x)):
            TargetGenes.append(genelist.get(x)[0])
        else:
            eprint('Missing ',x)
    
    TargetGenes=set(TargetGenes)
    TargetGenes=list(TargetGenes)
    
    TargetGenesEnsembl=set(TargetGenesEnsembl)
    TargetGenesEnsembl=list(TargetGenesEnsembl)
    
    eprint("Target gene list size =",len(TargetGenes))
    
    fd = open(Param_TargetFilename,"w")
    for g in TargetGenes:
        fd.write(g+"\n")
    fd.close()
    fd = open(Param_TargetFilename+"_EnsemblId","w")
    for g in TargetGenesEnsembl:
        fd.write(g+"\n")
    fd.close()
    
    

Missing  ENSG00000123009
Missing  ENSG00000165862
Target gene list size = 187


Regulatory network reconstruction using PathwayCommons SPARQL endpoint

In [10]:
# Algorithms

Aliases = dict()
for g in GeneAltNameToName:
    if not Aliases.get(GeneAltNameToName.get(g)):
        Aliases[GeneAltNameToName.get(g)] = list()
    Aliases[GeneAltNameToName.get(g)].append(g)



def GetTFControllers(EnsemblId):
    commonPCPrefixes = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX bp3: <http://www.biopax.org/release/biopax-level3.owl#>
PREFIX taxon: <http://identifiers.org/taxonomy/>
PREFIX reactome: <http://identifiers.org/reactome/>
PREFIX release: <http://www.reactome.org/biopax/49/48887#>

PREFIX up: <http://purl.uniprot.org/core/> 
PREFIX uniprot: <http://purl.uniprot.org/uniprot/>
PREFIX bp: <http://www.biopax.org/release/biopax-level3.owl#>
PREFIX chebi: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX obo2: <http://purl.obolibrary.org/obo#>
"""

    Request = """
SELECT ?tempReac ?type ?controlledName ?controllerName ?source WHERE{ 
    FILTER( (?controlledName = 'Transcription of %%GENETARGETNAME%%'^^<http://www.w3.org/2001/XMLSchema#string>)
        and (?controllerName != '%%GENETARGETNAME%%')
        and (?source != 'mirtarbase') ) .
    ?tempReac a bp:TemplateReactionRegulation .
    ?tempReac bp:displayName ?reacName ; 
        bp:controlled ?controlled ; 
        bp:controller ?controller ; 
        bp:controlType ?type ; 
        bp:dataSource ?source .
    ?controlled bp:displayName ?controlledName .
    ?controller bp:displayName ?controllerName . }
GROUP BY ?controlledName ?controllerName
"""
    RequestToBePerformed=commonPCPrefixes+Request.replace("%%GENETARGETNAME%%",EnsemblId)
    sparql = SPARQLWrapper(PC_Endpoint)
    sparql.setQuery(RequestToBePerformed)
    sparql.setReturnFormat(JSON)
    query = sparql.query().convert()
    results=list()
    listofgenes=list()
    ListBindings=query.get('results').get('bindings')
    for l in ListBindings:
        controllerName = (l.get('controllerName').get('value'))
        controlType= (l.get('type').get('value'))
        rule = controllerName+"_"+controlType+"_"+EnsemblId
        if (rule not in results):
            results.append(rule)
        if (controllerName not in listofgenes):
            listofgenes.append(controllerName)
    return results,listofgenes

def getParentGraph(l,limit=30):
    i=0
    allgenes=dict()
    for g in l:
        allgenes[g]=1
    listofrelations=list()
    unmapped=list()
    while (i<len(l) and i<limit):
        gene=l[i]
        L=Aliases.get(GeneAltNameToName.get(gene))
        relations=list()
        newgenes=list()
        for g in L:
            a,b=GetTFControllers(g)
            relations=relations+a
            newgenes=newgenes+b
        
        relations=list(set(relations))
        newgenes=list(set(newgenes))
#        relations,newgenes = GetTFControllers(GeneAltNameToName.get(gene))
        if (len(relations) == 0):
            unmapped.append(gene)
        listofrelations+=relations
        for g in newgenes:
            if (allgenes.get(g) != 1): # (g not in l):
                l.append(g)
                allgenes[g]=1
        i+=1
    return listofrelations,l,unmapped




# Improved version

def GetTFControllersList(EnsemblId):
    commonPCPrefixes = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX bp3: <http://www.biopax.org/release/biopax-level3.owl#>
PREFIX taxon: <http://identifiers.org/taxonomy/>
PREFIX reactome: <http://identifiers.org/reactome/>
PREFIX release: <http://www.reactome.org/biopax/49/48887#>

PREFIX up: <http://purl.uniprot.org/core/> 
PREFIX uniprot: <http://purl.uniprot.org/uniprot/>
PREFIX bp: <http://www.biopax.org/release/biopax-level3.owl#>
PREFIX chebi: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX obo2: <http://purl.obolibrary.org/obo#>
"""
# SELECT ?tempReac ?type ?controlledName ?controllerName ?source FROM <http://PC> 

    Request = """
SELECT ?tempReac ?type ?controlledName ?controllerName ?source
WHERE{ 
    FILTER(
    ("""
    for i in range(0,len(EnsemblId)-1):
        EnsemblId[i]=EnsemblId[i].replace("'","%27")
        Request=Request+"""(?controlledName = 'Transcription of %%GENETARGETNAME%%'^^<http://www.w3.org/2001/XMLSchema#string>) or 
        """.replace("%%GENETARGETNAME%%",EnsemblId[i])
#        Request=Request+"""(?controllerName = '%%GENETARGETNAME%%'^^<http://www.w3.org/2001/XMLSchema#string>) or 
#        """.replace("%%GENETARGETNAME%%",EnsemblId[i])
#    Request=Request+"""(?controllerName = '%%GENETARGETNAME%%'^^<http://www.w3.org/2001/XMLSchema#string>) or 
#    """.replace("%%GENETARGETNAME%%",EnsemblId[len(EnsemblId)-1])
    Request=Request+"(?controlledName = 'Transcription of %%GENETARGETNAME%%'^^<http://www.w3.org/2001/XMLSchema#string>)".replace("%%GENETARGETNAME%%",EnsemblId[len(EnsemblId)-1])
    Request=Request+""")
        and (?source != 'mirtarbase') ) .
    ?tempReac a bp:TemplateReactionRegulation .
    ?tempReac bp:displayName ?reacName ; 
        bp:controlled ?controlled ; 
        bp:controller ?controller ; 
        bp:controlType ?type ; 
        bp:dataSource ?source .
    ?controlled bp:displayName ?controlledName .
    ?controller bp:displayName ?controllerName . }
    
GROUP BY ?controlledName ?controllerName
"""
#    print(Request)
#    RequestToBePerformed=commonPCPrefixes+Request.replace("%%GENETARGETNAME%%",EnsemblId)
    RequestToBePerformed=commonPCPrefixes+Request
    sparql = SPARQLWrapper(PC_Endpoint)
    sparql.setQuery(RequestToBePerformed)
    sparql.setReturnFormat(JSON)
    sparql.setMethod("POST")
    query = sparql.query().convert()
    results=list()
    listofgenes=list()
    ListBindings=query.get('results').get('bindings')
    for l in ListBindings:
        controllerName = (l.get('controllerName').get('value'))
        controlType= (l.get('type').get('value'))
        controlledGene=(l.get('controlledName').get('value')).replace("Transcription of ","")
        rule = controllerName+"_"+controlType +"_"+GeneAltNameToName.get(controlledGene)
        if (rule not in results):
            results.append(rule)
        if (controllerName not in listofgenes):
            listofgenes.append(controllerName)
    return results,listofgenes

# New Improved version

def GetTFControllersList(EnsemblId):
    commonPCPrefixes = """
PREFIX bp: <http://www.biopax.org/release/biopax-level3.owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
"""

    Request = """
SELECT ?tempReac ?type ?controlledName ?controllerName ?source ?controllerType ?controlledType
WHERE{ 
    FILTER (
    ("""
    for i in range(0,len(EnsemblId)-1):
        EnsemblId[i]=EnsemblId[i].replace("'","%27")
#        Request=Request+"""((?controlledName = '%%GENETARGETNAME%%'^^xsd:string) and (?controllerName != '%%GENETARGETNAME%%'^^xsd:string) ) or 
        Request=Request+"""((?controlledName = '%%GENETARGETNAME%%'^^xsd:string) ) or 
        """.replace("%%GENETARGETNAME%%",EnsemblId[i])

#    Request=Request+"((?controlledName = '%%GENETARGETNAME%%'^^xsd:string) and (?controllerName != '%%GENETARGETNAME%%'^^xsd:string) )".replace("%%GENETARGETNAME%%",EnsemblId[len(EnsemblId)-1])
    Request=Request+"((?controlledName = '%%GENETARGETNAME%%'^^xsd:string) )".replace("%%GENETARGETNAME%%",EnsemblId[len(EnsemblId)-1])
    Request=Request+""")
        and (str(?source) != \"http://pathwaycommons.org/pc2/mirtarbase\") 
        and ((str(?controlledType) = "http://www.biopax.org/release/biopax-level3.owl#Protein") or (str(?controlledType) = "http://www.biopax.org/release/biopax-level3.owl#Rna"))
        and ((str(?controllerType) = "http://www.biopax.org/release/biopax-level3.owl#Protein") or (str(?controllerType) = "http://www.biopax.org/release/biopax-level3.owl#Rna"))
) .
?tempReac a bp:TemplateReactionRegulation .
?tempReac rdf:type ?ctype ; bp:controlled ?controlled ; bp:controller ?controller ; bp:controlType ?type ; bp:dataSource ?source .
?controlled bp:participant ?participant ; bp:dataSource ?controlledsource .
?participant bp:displayName ?controlledName; rdf:type ?controlledType .
?controller bp:displayName ?controllerName ; rdf:type ?controllerType ; bp:dataSource ?controllersource .
}    
GROUP BY ?controlledName ?controllerName
"""
#    print(Request)
#    RequestToBePerformed=commonPCPrefixes+Request.replace("%%GENETARGETNAME%%",EnsemblId)
    RequestToBePerformed=commonPCPrefixes+Request
    sparql = SPARQLWrapper(PC_Endpoint)
    sparql.setQuery(RequestToBePerformed)
    sparql.setReturnFormat(JSON)
    sparql.setMethod("POST")
    query = sparql.query().convert()
    results=list()
    listofgenes=list()
    ListBindings=query.get('results').get('bindings')
    for l in ListBindings:
        controllerName = (l.get('controllerName').get('value')).replace(" protein","").replace(" alternative form","")
        controlType= (l.get('type').get('value'))
        controlledGene=(l.get('controlledName').get('value')).replace("Transcription of ","")
        rule = controllerName+"_"+controlType +"_"+GeneAltNameToName.get(controlledGene)
        if (rule not in results):
            results.append(rule)
        if (controllerName not in listofgenes):
            listofgenes.append(controllerName)
    return results,listofgenes


"""
PREFIX bp: <http://www.biopax.org/release/biopax-level3.owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#\n"
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
CONSTRUCT {
 ?tempReac rdf:type ?type ; bp:controlled ?controlled ; bp:controller ?controller ; bp:dataSource ?source ; bp:controlType ?controlType .
  ?controlled a ?controlledType ; bp:displayName ?controlledName ; bp:dataSource ?controlledsource .
  ?controller a ?controllerType ; bp:displayName ?controllerName ; bp:dataSource ?controllersource .
} WHERE{ 
  FILTER( (?controlledName = \""+TF+"\"^^xsd:string)
  && (?controllerName != \""+TF+"\"^^xsd:string)
  && (str(?source) != \"http://pathwaycommons.org/pc2/mirtarbase\") ) .
?tempReac a bp:TemplateReactionRegulation .
?tempReac rdf:type ?type ; bp:controlled ?controlled ; bp:controller ?controller ; bp:controlType ?controlType ; bp:dataSource ?source .
?controlled bp:participant ?participant ; bp:dataSource ?controlledsource .
?participant bp:displayName ?controlledName; rdf:type ?controlledType .
?controller bp:displayName ?controllerName ; rdf:type ?controllerType ; bp:dataSource ?controllersource .
}
"""





def getParentGraphList(l,limit=30):
    pas=50
    i=0
    allgenes=dict()
    for g in l:
        allgenes[g]=1
    listofrelations=list()
    unmapped=list()
    while (i<len(l) and i<limit):
        eprint("Treated ",i," genes, still ",(len(l)-i)," limit = ",limit)
        L=list()
        for j in range(0,pas):
            if (i+j<len(l)):
                gene=l[i+j]
                if (Aliases.get(GeneAltNameToName.get(gene)) != None):
                    L=L+Aliases.get(GeneAltNameToName.get(gene))
        relations,newgenes=GetTFControllersList(L)
        if (len(relations) == 0):
            unmapped.append(gene)
        listofrelations+=relations
        for g in newgenes:
            g=GeneAltNameToName.get(g)
            if (g not in l): #(allgenes.get(g) != 1): # 
                l.append(g)
                allgenes[g]=1
        i+=pas
    return listofrelations,l,unmapped










In [11]:
eprint("Entering : RRG reconstruction")


## DEBUG ONLY
begin = timer()
## END DEBUG ONLY

if (Param_DoReconstruction):
    TargetGenesName = TargetGenes #[EnsemblToGeneNames.get(x) for x in TargetGenes if (len(x) > 2)]
    
    a,b,u=getParentGraphList(TargetGenesName,50000)
    
    ## DEBUG ONLY
    end = timer()
    eprint("Execution time for RRG reconstruction = ",end-begin,"seconds")
    ## END DEBUG ONLY
    
    eprint('Reconstructed graph contains',len(b),'genes,',len(a),'interactions')
    
    
    # write the file for compatibility purpose
    
    fd = open(Param_RRGFilename,"w")
    for r in a:
        relation=r.split("_")
#        if (relation[1] != "ACTIVATION"):
#        eprint(relation)
        fd.write(relation[0]+";"+relation[2]+"\n")
    fd.close()


Entering : RRG reconstruction
Treated  0  genes, still  187  limit =  50000
Treated  50  genes, still  203  limit =  50000
Treated  100  genes, still  200  limit =  50000
Treated  150  genes, still  187  limit =  50000
Treated  200  genes, still  180  limit =  50000
Treated  250  genes, still  149  limit =  50000
Treated  300  genes, still  105  limit =  50000
Treated  350  genes, still  55  limit =  50000
Treated  400  genes, still  8  limit =  50000
Execution time for RRG reconstruction =  68.81937293299961 seconds
Reconstructed graph contains 409 genes, 4155 interactions


In [12]:
endtotal = timer()
eprint("Execution completed in",endtotal-begintotal,"seconds")

Execution completed in 113.84074216299996 seconds


In [13]:
tg = Allmeds.get(Param_Drug).get('TargetsGenes')

In [14]:
possibleTargets=dict()
for m in Allmeds:
    tg = Allmeds.get(m).get('TargetsGenes')
    malist = [GeneAltNameToName.get(x) for x in b if GeneAltNameToName.get(x) in tg]
    for g in malist:
        if not possibleTargets.get(g):
            possibleTargets[g]=list()
        possibleTargets[g].append(m)
#    if (len(malist)>0):
#        print(m,malist)

In [15]:
possibleTargets

{'CEL': ['Xenical (TN)'],
 'DGAT1': ['Pradigastat sodium (USAN)', 'Pradigastat (USAN)'],
 'IFNG': ['Fontolizumab (USAN/INN)'],
 'IL1B': ['Ilaris (TN)', 'Gevokizumab (USAN/INN)'],
 'NPC1L1': ['Zetia (TN)'],
 'NR1H4': ['Chenix (TN)', 'Ocaliva (TN)', 'Turofexorate isopropyl (USAN/INN)'],
 'NR3C1': ['Clobetasone (INN)',
  'Mometasone (INN)',
  'Haldrone (TN)',
  'Loteprednol (INN)',
  'Dexamedium [veterinary] (TN)',
  'Vanos (TN)',
  'Sintisone (TN)',
  'Elocon (TN)',
  'Afusona (TN)',
  'Tixocortol (INN)',
  'Triamcinolone acetonide sodium phosphate (USAN)',
  'Omnaris (TN)',
  'Olux (TN)',
  'Cortifoam (TN)',
  'Kohakusanin (TN)',
  'Veramyst (TN)',
  'Vanceril (TN)',
  'Uticort (TN)',
  'Methylprednisolone suleptanate (USAN/INN)',
  'Beclometason (TN)',
  'Betamethasone acetate (JAN/USP)',
  'Delvosteron [veterinary] (TN)',
  'Cordran (TN)',
  'Prednisone 21-acetate',
  'Methylprednisolone sodium phosphate (USAN)',
  'Depo-medrol (TN)',
  'Hydrocortisone succinate (JP17)',
  'Hydrocorti

In [16]:

import json
with open("../tmp/PossibleTargets.json", 'w') as outfile:
    json.dump(possibleTargets, outfile)


In [17]:
ListReg=list()
for g in regulators:
    if (EnsemblToGeneNamesList.get(g) != None):
        ListReg=ListReg+EnsemblToGeneNamesList.get(g)

In [18]:
len(ListReg)

194

In [19]:
possibleTargetsAvant=dict()
for m in Allmeds:
    tg = Allmeds.get(m).get('TargetsGenes')
    malist = [GeneAltNameToName.get(x) for x in ListReg if GeneAltNameToName.get(x) in tg]
    for g in malist:
        if not possibleTargetsAvant.get(g):
            possibleTargetsAvant[g]=list()
        possibleTargetsAvant[g].append(m)

In [20]:
possibleTargetsAvant

{'CEL': ['Xenical (TN)'],
 'DGAT1': ['Pradigastat sodium (USAN)', 'Pradigastat (USAN)'],
 'NPC1L1': ['Zetia (TN)'],
 'PNLIP': ['Xenical (TN)'],
 'SLC22A1': ['Metformin (USAN/INN)', 'Glucophage (TN)', 'Eloxatin (TN)']}

In [21]:
print("Le nombre total de cibles potentielles après reconstruction est",len(possibleTargets))

Le nombre total de cibles potentielles après reconstruction est 14


In [22]:
print("Le nombre total de cibles potentielles avant reconstruction est",len(possibleTargetsAvant))

Le nombre total de cibles potentielles avant reconstruction est 5


In [23]:
Drugs=list()
for t in possibleTargets:
    lg=possibleTargets.get(t)
    for g in lg:
        Drugs.append(g)
        
Drugs=list(set(Drugs))

In [24]:
len(Drugs)

175

In [25]:
import networkx as nx

# construction du graphe de régulation

G=nx.DiGraph()

for i in a:
    source,typ,target=i.split("_")
    G.add_edge(source,target)

print("Le réseau de régulation possède",len(G.nodes()),"gènes")

print("Le réseau de régulation possède",len(G.edges()),"interactions")

Le réseau de régulation possède 370 gènes
Le réseau de régulation possède 4155 interactions


In [26]:
# essai : ici, on prend l'ensemble des drogues de KeggDrug et on regarde tous les gènes impactés par au moins une drogue

L=list()
for g in possibleTargets:
    L.append(g)

L=L+nx.algorithms.node_boundary(G,L)

In [27]:
# G2 sera le sous-graphe induit par L (i.e., le sous-graphe de G dont tous les sommets sont dans L)

G2=G.copy()
G2.remove_nodes_from(n for n in G.nodes() if n not in L)

print("Le sous-réseau de régulation impacté possède",len(G2.nodes()),"gènes")
print("Le sous-réseau de régulation impacté possède",len(G2.edges()),"interactions")

Le sous-réseau de régulation impacté possède 187 gènes
Le sous-réseau de régulation impacté possède 2078 interactions


In [28]:
GenesCibles=list()
for e in regulators:
    LR=EnsemblToGeneNamesList.get(e)
    if (LR != None):
        GenesCibles=GenesCibles+LR

len(GenesCibles)

194

In [29]:
MetFlux["NEFA"]=9.435
MetFlux["glucose"]=61.967
MetFlux["linoleate"]=-1.4771
MetFlux["histidine"]=-0.42186
MetFlux["isoleucine"]=-4.5346
MetFlux["leucine"]=-4.8616
MetFlux["lysine"]=-3.7669
MetFlux["methionine"]=-1.512
MetFlux["phenylalanine"]=-0.42186
MetFlux["threonine"]=-2.1126
MetFlux["tryptophan"]=-0.42186
MetFlux["valine"]=-2.5502
MetFlux["H2O"]=0.012347
MetFlux["O2"]=-47.1389
MetFlux["CO2"]=-32.3697
MetFlux["alanine"]=-2.0184
MetFlux["asparagine"]=1.24
MetFlux["glutamine"]=18.1648
MetFlux["tyrosine"]=1.7553
MetFlux["cysteine"]=-0.66829
MetFlux["arginine"]=2.7083
MetFlux["glycine"]=1.4082
MetFlux["proline"]=0.20996
MetFlux["serine"]=5.5838
MetFlux["aspartate"]=1.8467
MetFlux["glutamate"]=1.411
MetFlux["Pi"]=-0.012347
MetFlux["NH3"]=-2.8121
MetFlux["HCO3-"]=-0.80866
MetFlux["choline"]=0.42717
MetFlux["ethanolamine"]=-4.8705
MetFlux["acetate"]=-47.4465
MetFlux["ornithine"]=-30.0037
MetFlux["5-formyl-THF"]=0.24872
MetFlux["acetoacetate"]=-0.42157
MetFlux["L-lactate"]=-41.7375
MetFlux["fructose"]=-38.4359
MetFlux["ADP-ribose"]=0.012519
MetFlux["folate"]=0.0022124
MetFlux["homoserine"]=0.6349
MetFlux["citrulline"]=0.082847
MetFlux["2-hydroxybutyrate"]=-3.4158
MetFlux["5-methyl-THF"]=-0.66829
MetFlux["adenine"]=-0.00093523
MetFlux["adenosine"]=-0.00040792
MetFlux["ADP"]=0.0012202
MetFlux["AKG"]=-0.0015523
MetFlux["AMP"]=-0.0012202
MetFlux["CMP"]=-0.0015891
MetFlux["cytidine"]=0.0021187
MetFlux["deoxycytidine"]=0.0018937
MetFlux["deoxyuridine"]=0.0020755
MetFlux["GDP"]=0.0020071
MetFlux["GMP"]=-0.00063611
MetFlux["hypoxanthine"]=0.021934
MetFlux["inosine"]=-0.00024905
MetFlux["THF"]=0.41736
MetFlux["thymine"]=-0.000079265
MetFlux["thymidine"]=0.0015529
MetFlux["UDP"]=0.0045704
MetFlux["UMP"]=-0.0045704
MetFlux["uracil"]=-0.0084013
MetFlux["urea"]=-2.3693
MetFlux["uridine"]=0.0068026
MetFlux["UTP"]=0.61997
MetFlux["XTP"]=0.28912
MetFlux["ADP-glucose"]=0.31787
MetFlux["D-glucitol"]=-0.014345


NameError: name 'MetFlux' is not defined

In [ ]:
model = cobra.io.read_sbml_model('/Users/jeremiebourdon/Downloads/Hep-G2.xml')


MetFlux=dict()
MetFlux["propanoate"]=0.000078515
MetFlux["NEFA"]=9.435
MetFlux["glucose"]=61.967
MetFlux["linoleate"]=-1.4771
MetFlux["histidine"]=-0.42186
MetFlux["isoleucine"]=-4.5346
MetFlux["leucine"]=-4.8616
MetFlux["lysine"]=-3.7669
MetFlux["methionine"]=-1.512
MetFlux["phenylalanine"]=-0.42186
MetFlux["threonine"]=-2.1126
MetFlux["tryptophan"]=-0.42186
MetFlux["valine"]=-2.5502
MetFlux["H2O"]=0.012347
MetFlux["O2"]=-47.1389
MetFlux["CO2"]=-32.3697
MetFlux["alanine"]=-2.0184
MetFlux["asparagine"]=1.24
MetFlux["glutamine"]=18.1648
#MetFlux["tyrosine"]=1.7553
MetFlux["cysteine"]=-0.66829
MetFlux["arginine"]=2.7083
MetFlux["glycine"]=1.4082
MetFlux["proline"]=0.20996
MetFlux["serine"]=5.5838
#MetFlux["aspartate"]=1.8467
MetFlux["glutamate"]=1.411
MetFlux["Pi"]=-0.012347
MetFlux["NH3"]=-2.8121
#MetFlux["HCO3-"]=-0.80866
MetFlux["choline"]=0.42717
MetFlux["ethanolamine"]=-4.8705
MetFlux["acetate"]=-47.4465
MetFlux["ornithine"]=-30.0037
#MetFlux["5-formyl-THF"]=0.24872
MetFlux["acetoacetate"]=-0.42157
MetFlux["L-lactate"]=-41.7375
MetFlux["fructose"]=-38.4359
#MetFlux["ADP-ribose"]=0.012519
MetFlux["folate"]=0.0022124
#MetFlux["homoserine"]=0.6349
#MetFlux["citrulline"]=0.082847
MetFlux["2-hydroxybutyrate"]=-3.4158
MetFlux["5-methyl-THF"]=-0.66829
MetFlux["adenine"]=-0.00093523
MetFlux["adenosine"]=-0.00040792
MetFlux["ADP"]=0.0012202
MetFlux["AKG"]=-0.0015523
MetFlux["AMP"]=-0.0012202
MetFlux["CMP"]=-0.0015891
MetFlux["cytidine"]=0.0021187
MetFlux["deoxycytidine"]=0.0018937
MetFlux["deoxyuridine"]=0.0020755
MetFlux["GDP"]=0.0020071
MetFlux["GMP"]=-0.00063611
MetFlux["hypoxanthine"]=0.021934
MetFlux["inosine"]=-0.00024905
MetFlux["THF"]=0.41736
MetFlux["thymine"]=-0.000079265
MetFlux["thymidine"]=0.0015529
MetFlux["UDP"]=0.0045704
MetFlux["UMP"]=-0.0045704
MetFlux["uracil"]=-0.0084013
#MetFlux["urea"]=-2.3693
MetFlux["uridine"]=0.0068026
MetFlux["UTP"]=0.61997
#MetFlux["XTP"]=0.28912
#MetFlux["ADP-glucose"]=0.31787
MetFlux["D-glucitol"]=-0.014345

modelC=model.copy()
for met in MetFlux:
    find=0
    for m in modelC.metabolites:
        if (m.name == met and m.compartment == "s"):
            for r in m.reactions:
                if (len(r.reactants) == 0):        
                    #print(m.id+":"+m.name, r.id, r.build_reaction_string(), r.lower_bound, r.upper_bound, -MetFlux[met])
                    diff=0.1*abs(MetFlux[met])
                    r.lower_bound=MetFlux[met]-diff
                    r.upper_bound=MetFlux[met]+diff
                    find+=1
    if find == 0:
        print("no match for ",met)
modelC.reactions.biomass_components.objective_coefficient = -1
fmin=modelC.optimize()
modelC.reactions.biomass_components.objective_coefficient = 1
fmax=modelC.optimize()    
print(met,":",fmin.f,fmax.f)
# print(model.compartments)

model.reactions.biomass_components.objective_coefficient=-1
print(model.reactions.biomass_components.build_reaction_string())
print(model.reactions.biomass_components.upper_bound)

model = cobra.io.read_sbml_model('/Users/jeremiebourdon/Downloads/liver_-_hepatocytes_2 4.xml')
model.optimize()

cobra.flux_analysis.flux_variability_analysis(model)